In [1]:
import numpy as np
from math import sqrt,log10
import pylab as plt
from scipy.interpolate import interp1d as interp
from scipy.optimize import brentq, minimize_scalar
from profiles import build_profile, build_kernel
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood
from fitter import MinuitFitter

%matplotlib inline

In [2]:
directory = '/home/andrea/Desktop/work/DWARF/dsphsim/'
rh = 0.04
D = 39.81
theta = 2*rh/D
dwarf_props = {'D':D, 'theta':theta, 'rt':np.inf}

## True values

In [3]:
dm_true = build_profile('NFW')
rvmax = 0.757
vmax = 10.4
rs = rvmax/2.163
rhos = 4.625 * (vmax/rs)**2 / (4*np.pi*4.302e-6)
dm_true.r0 = rs
dm_true.rho0 = rhos
Jtrue = log10(dm_true.Jfactor(**dwarf_props))
print 'rs = %.2f, rhos = %.2e, J = %g'%(rs,rhos,Jtrue)

rs = 0.35, rhos = 7.55e+07, J = 16.724


In [4]:
dm = build_profile('NFW')
st = build_profile('plummer',rh=rh) # Plummer Stellar profile
kr = build_kernel('constbeta') # isotropic kernel
Sigma = SphericalJeansDispersion(dm, st, kr, dwarf_props)

In [5]:
R, v = np.loadtxt(directory+'Ret2_data/dsph_002.txt',usecols=(5, 7),unpack=True)
vnan = np.isnan(v)
v = v[~vnan]
R = R[~vnan]
dv = np.zeros_like(v)

In [6]:
LL = GaussianLikelihood([R, v, dv, 0.], Sigma)
LL.set_free('dm_r0')
LL.set_free('dm_a')
LL.set_free('dm_b')
LL.set_free('dm_c')
LL.set_free('ker_beta')

In [7]:
LL.free_pars.keys()

['dm_r0', 'J', 'dm_a', 'dm_b', 'dm_c', 'ker_beta']

In [8]:
J_array = np.linspace(15,18,20)
L_arr = np.empty([0])
parameters = np.array(LL.free_pars.keys())
for par in parameters:
    parname = par.split('_')[-1]
    exec( '%s_arr = np.zeros_like(J_array)'%parname)

In [ ]:
for j,J in enumerate(J_array):
    M = MinuitFitter(LL)
    # J
    M.set_value('J',J)
    M.set_fixed('J')
    # r0
    M.set_value('dm_r0',rh*2.)
    M.set_error('dm_r0',0.01)
    # a
    M.set_value('dm_a',1.)
    M.set_error('dm_a',0.01)
    # b
    M.set_value('dm_b',3.)
    M.set_error('dm_b',0.01)
    # c
    M.set_value('dm_c',1.)
    M.set_error('dm_c',0.01)
    # beta
    M.set_value('ker_beta',0.)
    M.set_error('ker_beta',0.01)
    
    M.settings['print_level'] = 1
    M.set_minuit(**{'tol':1e7,'strategy':1})
    BF = M.migrad_min()
    
    valid = False
    while not valid:
        BF = M.migrad_min()
        valid = BF[0]['is_valid']
        if M.minuit.tol>1e10 and not valid: break
        M.minuit.tol *= 10
    else:
        L_arr = np.append(L_arr,BF[0]['fval'])
        for par in parameters:
            parname = par.split('_')[-1]
            indx = np.where( parameters==par )[0][0]
            exec( "{0}_arr[{1}] = BF[1][{2}]['value']".format(parname,j,indx))

/usr/lib/python2.7/dist-packages/scipy/integrate/quadpack.py:295: UserWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = +0.080000
           J = +15.000000
        dm_a = +1.000000
        dm_b = +3.000000
        dm_c = +1.000000
    ker_beta = -0.605961

  return self.minuit.migrad(**kwargs)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = -nan
           J = +15.000000
        dm_a = -nan
        dm_b = -nan
        dm_c = -nan
    ker_beta = -nan

  return self.minuit.migrad(**kwargs)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = +nan
           J = +15.000000
        dm_a = +nan
        dm_b = +nan
        dm_c = +nan
    ker_beta = +nan

  return self.

FCN = 1639.51520334 
 TOTAL NCALL = 131 
 NCALLS = 118 
 
 
 EDM = 215948.2046 
 GOAL EDM = 500.0 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,dm_r0,8.000000e-02,6.267606e-03,0.000000e+00,0.000000e+00,,,
2,J,1.500000e+01,1.000000e+00,0.000000e+00,0.000000e+00,,,FIXED
3,dm_a,1.000000e+00,5.208283e-02,0.000000e+00,0.000000e+00,,,
4,dm_b,3.000000e+00,3.059146e-01,0.000000e+00,0.000000e+00,,,
5,dm_c,1.000000e+00,4.196722e-02,0.000000e+00,0.000000e+00,,,
6,ker_beta,-3.996399e-01,1.204372e-01,0.000000e+00,0.000000e+00,,,


fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = -3.973379
           J = +15.000000
        dm_a = -32.745251
        dm_b = +202.120365
        dm_c = +28.445679
    ker_beta = -79.533388

  return self.minuit.migrad(**kwargs)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = -4.176048
           J = +15.000000
        dm_a = -34.432514
        dm_b = +212.076383
        dm_c = +29.817963
    ker_beta = -83.490076

  return self.minuit.migrad(**kwargs)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = -4.186181
           J = +15.000000
        dm_a = -34.516877
        dm_b = +212.574184
        dm_c = +29.886577
    ker_beta = -83.687910

  return self.minuit.migrad(**kwargs)
fitter.py:81: RuntimeWarning: fcn returns Nan
fcn is called with following arguments:
       dm_r0 = -4.186688
           J = +15.000000
        dm_a = -34.5

FCN = 1639.51520334 
 TOTAL NCALL = 178 
 NCALLS = 21 
 
 
 EDM = 215947.096165 
 GOAL EDM = 500.0 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 False 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 True 
 
 False

...

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,dm_r0,7.821392e-02,4.540707e-02,0.000000e+00,0.000000e+00,,,
2,J,1.721053e+01,1.000000e+00,0.000000e+00,0.000000e+00,,,FIXED
3,dm_a,9.680329e-01,3.763923e-01,0.000000e+00,0.000000e+00,,,
4,dm_b,3.017228e+00,1.290121e+00,0.000000e+00,0.000000e+00,,,
5,dm_c,1.008445e+00,9.364203e-02,0.000000e+00,0.000000e+00,,,
6,ker_beta,-1.354648e-02,9.783866e-02,0.000000e+00,0.000000e+00,,,


FCN = 1110.77972135 
 TOTAL NCALL = 128 
 NCALLS = 64 
 
 
 EDM = 2.34106314249 
 GOAL EDM = 500.0 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,dm_r0,7.524746e-02,4.649765e-02,0.000000e+00,0.000000e+00,,,
2,J,1.721053e+01,1.000000e+00,0.000000e+00,0.000000e+00,,,FIXED
3,dm_a,9.430100e-01,4.094280e-01,0.000000e+00,0.000000e+00,,,
4,dm_b,2.912556e+00,1.154759e+00,0.000000e+00,0.000000e+00,,,
5,dm_c,1.006786e+00,1.146980e-01,0.000000e+00,0.000000e+00,,,
6,ker_beta,-1.053539e-02,9.154300e-02,0.000000e+00,0.000000e+00,,,


FCN = 1116.91656736 
 TOTAL NCALL = 66 
 NCALLS = 66 
 
 
 EDM = 38.9682680345 
 GOAL EDM = 500.0 
 
 UP = 0.5 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,dm_r0,7.705727e-02,7.950934e-02,0.000000e+00,0.000000e+00,,,
2,J,1.736842e+01,1.000000e+00,0.000000e+00,0.000000e+00,,,FIXED
3,dm_a,9.683585e-01,7.388755e-01,0.000000e+00,0.000000e+00,,,
4,dm_b,3.021004e+00,2.570736e-01,0.000000e+00,0.000000e+00,,,
5,dm_c,1.004555e+00,6.008629e-02,0.000000e+00,0.000000e+00,,,
6,ker_beta,1.778417e-04,3.278428e-02,0.000000e+00,0.000000e+00,,,


In [ ]:
interp_L = interp(J_arr, L_arr-L_arr.min())
for par in parameters:
    parname = par.split('_')[-1]
    exec( 'interp_{p} = interp(J_arr, {p}_arr)'.format(p=parname))

In [ ]:
eval_Like_J = np.linspace(J_arr.min(), J_arr.max(), 1e3)
min_Like_J = interp_L(eval_Like_J).min()
Jmin = eval_Like_J[ np.where( interp_L(eval_Like_J) == min_Like_J )[0][0] ]
J_plt = np.linspace(J_arr.min(),J_arr.max())

In [ ]:
Jr = float(interp_r0(Jmin))
Ja = float(interp_a(Jmin))
Jb = float(interp_b(Jmin))
Jc = float(interp_c(Jmin))
Jbeta = float(interp_beta(Jmin))

In [ ]:
dm.r0 = Jr
dm.a = Ja
dm.b = Jb
dm.c = Jc
minrho = lambda rho : abs(Jmin - np.log10( dm.Jfactor(**dwarf_props) ) - 2*rho)
Jrho = float(10**minimize_scalar(minrho).x)

In [ ]:
print 'r = %.2f, rho = %.2e, a = %.2f, b = %.2f, c = %.2f'%(Jr, Jrho, Ja, Jb, Jc)
#print 'r = %.2f, rho = %.2e , c = %.2f'%(Jr, Jrho ,Jc)

In [ ]:
for i,c in enumerate([0.5,2.0,4.5]):
    exec('J%ssL = %.2f'%(i+1, brentq(lambda j : interp_L(j)-c, J_arr.min(), Jmin) - Jmin))
    exec('J%ssR = %.2f'%(i+1, brentq(lambda j : interp_L(j)-c, Jmin, J_arr.max()) - Jmin))

In [ ]:
plt.plot(J_plt, interp_L(J_plt))
plt.hlines(0, J_plt.min(), J_plt.max(), linestyles='dashed')
plt.plot(Jmin, 0, 'b^', markersize=10, label=r'$\mathcal{J}_{MLE}$ = %.2f'%Jmin)
plt.plot(16.72, 0, 'rd', markersize=6, label=r'$\mathcal{J}_{TRUE}$ = 16.72')
for i,(c,h) in enumerate(zip(['r','g','c'],[0.5,2.0,4.5])):
    JL, JR = eval('J%ssL'%str(i+1)), eval('J%ssR'%str(i+1))
    plt.hlines(h, J_plt.min(), J_plt.max(), linestyles='dashed', colors=c,
              label=r'$%i-\sigma$'%(i+1)+'\t'+'[%2s,%2s]'%(str(JL),str(JR)))
plt.legend(numpoints=1,fontsize=14).get_frame().set_facecolor('w')
plt.text(16.3, 7,'a = %.2f (1)'%Ja +'\n'+
                 'b = %.2f (3)'%Jb +'\n'+
                 'c = %.2f (1)'%Jc +'\n'+
                 r'$r_s$ = %.2f (0.35)'%Jr +'\n'+
                 r'$\rho_s$ = %.2e'%Jrho +'\n     (7.55e+07)')

plt.ylim(-0.5,12);
#plt.xlim(16.3,17.5);
#plt.savefig('results/LikeJ_rabc.png',format='png',dpi=300);